# * Adhoc : DTAC Target

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import oracledb
import xlrd

In [2]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

In [3]:
curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 6)

## Import data

In [5]:
# src_file = 'C:/Ruz/Pentaho/Jobs/Input/Target/ADHOC_RAW_DTAC_SALE_TARGET_AREA.xlsx'
src_file = 'C:\Ruz\Pentaho\Jobs\Input\Target\ADHOC_RAW_DTAC_SALE_TARGET_AREA.xlsx'

src_df = pd.read_excel(src_file, sheet_name='DTAC_Sales_Target', skiprows=2, usecols='N:AD', index_col=None) 
src_df = src_df[src_df['USE_FLAG']=='Y']
src_df.head(3)

,USE_FLAG,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD.1,METRIC_NAME.1,METRIC_VALUE,COMP_CD.1,ORGID_P,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,PROVINCE_ENG,REMARK
0,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,4194780.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),050,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
1,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,397642.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),016,"BKK : Bangkok Yai, Bangkok Noi, Bang Phlat",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
2,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,3269671.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),040,"BKK : Taling Chan, Phasi Charoen, Thawi Wattha...",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)


In [31]:
src_df

,USE_FLAG,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD.1,METRIC_NAME.1,METRIC_VALUE,COMP_CD.1,ORGID_P,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,PROVINCE_ENG,REMARK
0,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,4194780.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),050,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
1,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,397642.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),016,"BKK : Bangkok Yai, Bangkok Noi, Bang Phlat",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
2,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,3269671.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),040,"BKK : Taling Chan, Phasi Charoen, Thawi Wattha...",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
3,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,961776.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),049,"BKK : Thon Buri, Khlong San, Rat Burana, Thung...",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
4,Y,202401,NaN,DB1R000900,Prepaid Inflow M1 : DTAC,2145895.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),202,"NTB : Bang Kruai, Bang Yai",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16230,Y,202405,NaN,DB2R000500AK,Postpaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,P,UPC,GX8,Regional Management 8 (South),83Z,PHUKET,NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
16231,Y,202405,NaN,DB2R000500AK,Postpaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,P,UPC,GX8,Regional Management 8 (South),86X,"RANONG, CHUMPHON",NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
16232,Y,202405,NaN,DB2R000500AK,Postpaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,P,UPC,GX8,Regional Management 8 (South),90Z,SONGKHLA,NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)
16233,Y,202405,NaN,DB2R000500AK,Postpaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,P,UPC,GX8,Regional Management 8 (South),84Z,SURAT THANI,NaN,NaN,NaN,DTAC Sales target Jan-May 2024(Ref : K.Voraphon)


In [ ]:
''' Rawdata '''

data_src = f"../data/raw/{op_file_name}.csv"
df = pd.read_csv(data_src, low_memory=False)

cols = ['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'CENTER', 'METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 
        'ACTUAL_AS_OF', 'AGG_TYPE', 'RR_IND', 'GRY_IND', 'UOM', 'PERIOD', 'ACTUAL_SNAP', 'ACTUAL_AGG', 'TARGET_SNAP', 'TARGET_AGG', 'BASELINE_SNAP', 'BASELINE_AGG', 'PPN_TM']

df = df[cols]

''' Filters '''
df = df[df['TM_KEY_YR'] == 2024] # YEAR filter
# df = df[df['TM_KEY_MTH'] >= 202401] # MONTH filter

df.tail()

# ''' Fix Metric CD list '''

# ''' Fix Agg Column Name(Total Revenue) '''
# bg_col_list = ['POST_DTAC', 'POST_TMH', 'PRE_DTAC', 'PRE_TMH', 'TOL', 'TVS']

### AKPIPRD >>>

In [4]:
# AKPIPRD

# conn = oracledb.connect(user=AKPIPRD_user, password=AKPIPRD_pwd, dsn=f'{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}')
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'{AKPIPRD_db} : Connected')

cur = conn.cursor()

v_date = (curr_dt, )

query = """
    SELECT *
    FROM (
        SELECT TM_KEY_DAY, METRIC_CD, METRIC_NAME, METRIC_VALUE, COMP_CD, VERSION, AREA_NO, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK
            , ROW_NUMBER() OVER(PARTITION BY METRIC_CD, AREA_CD, SUBSTR(TM_KEY_DAY,1,6) ORDER BY TM_KEY_DAY DESC) END_MTH_IND
        FROM AUTOKPI.VW_ACTUAL_AREA_DAILY 
        WHERE METRIC_CD LIKE 'VIN%'
        AND TM_KEY_DAY >= 20240101
        AND AREA_TYPE IN ('P', 'G', 'Z')
    ) TMP
    --WHERE END_MTH_IND = 1
    WHERE TM_KEY_DAY LIKE '%01'
    ORDER BY TM_KEY_DAY, METRIC_CD, AREA_NO, AREA_CD 
"""

try:
    print(f'\nExecute query as of {v_date[0]}...')
    cur.execute(query)
    rows = cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
    print(f'\ndf : {src_df.shape[0]} rows, {src_df.shape[1]} columns\nDone !!!')

except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')

finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')

AKPIPRD : Connected

Execute query as of 2024-06-06...

df : 576 rows, 13 columns
Done !!!

AKPIPRD : Disconnected


In [5]:
src_df.tail()

,TM_KEY_DAY,METRIC_CD,METRIC_NAME,METRIC_VALUE,COMP_CD,VERSION,AREA_NO,AREA_CD,AREA_DESC,AREA_TYPE,LOAD_DATE,REMARK,END_MTH_IND
571,20240601,VIN00024,Broadband Subs Share : CAT,2.92,TRUE,A,5,GX4,Regional Management 4 (North),G,2024-06-04 14:38:20.788865,None,9
572,20240601,VIN00024,Broadband Subs Share : CAT,2.78,TRUE,A,5,GX5,Regional Management 5 (Northeast 1),G,2024-06-04 14:38:20.788865,None,9
573,20240601,VIN00024,Broadband Subs Share : CAT,1.84,TRUE,A,5,GX6,Regional Management 6 (Northeast 2),G,2024-06-04 14:38:20.788865,None,9
574,20240601,VIN00024,Broadband Subs Share : CAT,2.67,TRUE,A,5,GX7,Regional Management 7 (Central West),G,2024-06-04 14:38:20.788865,None,9
575,20240601,VIN00024,Broadband Subs Share : CAT,3.90,TRUE,A,5,GX8,Regional Management 8 (South),G,2024-06-04 14:38:20.788865,None,9


## Reconcile

In [8]:
# AKPIPRD

# src_df[src_df['TM_KEY_DAY']==20240601].reset_index(drop=True)
src_df[(src_df['TM_KEY_DAY']==20240501) & (src_df['METRIC_CD']=='VIN00020')].sort_values(['TM_KEY_DAY', 'METRIC_CD', 'AREA_NO', 'AREA_CD']).reset_index(drop=True)

,TM_KEY_DAY,METRIC_CD,METRIC_NAME,METRIC_VALUE,COMP_CD,VERSION,AREA_NO,AREA_CD,AREA_DESC,AREA_TYPE,LOAD_DATE,REMARK,END_MTH_IND
0,20240501,VIN00020,Broadband Subs Share : TOL,36.52,TRUE,A,1,P,Nationwide,P,2024-06-04 14:14:53.206419,None,31
1,20240501,VIN00020,Broadband Subs Share : TOL,48.82,TRUE,A,2,BMA,BMA,Z,2024-06-04 14:14:53.206419,None,31
2,20240501,VIN00020,Broadband Subs Share : TOL,30.38,TRUE,A,2,UPC,UPC,Z,2024-06-04 14:14:53.206419,None,31
3,20240501,VIN00020,Broadband Subs Share : TOL,32.59,TRUE,A,3,EAST,East,Z,2024-06-04 14:14:53.206419,None,31
4,20240501,VIN00020,Broadband Subs Share : TOL,50.31,TRUE,A,4,10,Bangkok,Z,2024-06-04 14:14:53.206419,None,31
5,20240501,VIN00020,Broadband Subs Share : TOL,47.47,TRUE,A,4,11,Samut Prakan,Z,2024-06-04 14:14:53.206419,None,31
6,20240501,VIN00020,Broadband Subs Share : TOL,46.30,TRUE,A,4,12,Nonthaburi,Z,2024-06-04 14:14:53.206419,None,31
7,20240501,VIN00020,Broadband Subs Share : TOL,45.35,TRUE,A,4,13,Pathum Thani,Z,2024-06-04 14:14:53.206419,None,31
8,20240501,VIN00020,Broadband Subs Share : TOL,47.87,TRUE,A,5,GX1,Deputy CGO & Regional Management 1 (BMA-West),G,2024-06-04 14:14:53.206419,None,31
9,20240501,VIN00020,Broadband Subs Share : TOL,50.05,TRUE,A,5,GX2,Regional Management 2 (BMA-East),G,2024-06-04 14:14:53.206419,None,31


In [9]:
# TDMDBPR

# df[df['TM_KEY_DAY']==20240501].reset_index(drop=True)
df[(df['TM_KEY_DAY']==20240501) & (df['METRIC_CD']=='VIN00020')].sort_values(['TM_KEY_DAY', 'METRIC_CD', 'AREA_NO', 'AREA_CD']).reset_index(drop=True)

,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,ACTUAL_AGG,PPN_TM
0,20240501,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.52,36.52,2024-06-06 10:34:07
1,20240501,VIN00020,Broadband Subs Share : TOL,2,Z,BMA,BMA,48.82,48.82,2024-06-06 10:34:07
2,20240501,VIN00020,Broadband Subs Share : TOL,2,Z,UPC,UPC,30.38,30.38,2024-06-06 10:34:07
3,20240501,VIN00020,Broadband Subs Share : TOL,3,Z,EAST,EAST,32.59,32.59,2024-06-06 10:34:07
4,20240501,VIN00020,Broadband Subs Share : TOL,4,Z,10,Bangkok,50.31,50.31,2024-06-06 10:34:07
5,20240501,VIN00020,Broadband Subs Share : TOL,4,Z,11,Samut Prakan,47.47,47.47,2024-06-06 10:34:07
6,20240501,VIN00020,Broadband Subs Share : TOL,4,Z,12,Nonthaburi,46.30,46.30,2024-06-06 10:34:07
7,20240501,VIN00020,Broadband Subs Share : TOL,4,Z,13,Pathum Thani,45.35,45.35,2024-06-06 10:34:07
8,20240501,VIN00020,Broadband Subs Share : TOL,5,G,GX1,Regional Management 1 (BMA : West),47.87,47.87,2024-06-06 10:34:07
9,20240501,VIN00020,Broadband Subs Share : TOL,5,G,GX2,Regional Management 2 (BMA : East),50.05,50.05,2024-06-06 10:34:07


In [10]:
df[(df['METRIC_CD']=='VIN00020') & (df['AREA_CD']=='P')].sort_values(['TM_KEY_DAY', 'METRIC_CD', 'AREA_NO', 'AREA_CD']).reset_index(drop=True)

,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,ACTUAL_AGG,PPN_TM
0,20240101,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.49,36.49,2024-06-06 10:34:07
1,20240201,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.50,36.50,2024-06-06 10:34:07
2,20240301,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.52,36.52,2024-06-06 10:34:07
3,20240401,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.49,36.49,2024-06-06 10:34:07
4,20240501,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.52,36.52,2024-06-06 10:34:07
5,20240601,VIN00020,Broadband Subs Share : TOL,1,P,P,Nationwide,36.52,36.52,2024-06-06 10:34:07


## Generate Output file

In [11]:
# df.to_csv(f'{op_raw_file}', index=False, encoding='utf-8')

# print(f'\n"{op_raw_file}" is generated')